In [50]:
import requests
import time
import os, json
import csv
import concurrent.futures
# from functools import partial
import pandas as pd

In [51]:
# url = "https://refract.fosfor.com/magiccodermodel/6188a3e2-bc43-444e-bfaa-8159fca71b3a/score"
url = "http://vllm-serving-service:8888/v1/completions"
# lltoken = "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJ4WTdTd3k5UE1xaXRDQmNSMm5qcVl6bmoxS3NqZzV3TmdOV0xDVzdyUkhvIn0.eyJleHAiOjE3MzA4ODM3MzgsImlhdCI6MTY5OTI2MTMzOCwiYXV0aF90aW1lIjoxNjk5MjUyMDQ0LCJqdGkiOiJmN2EzMzQwYy1kNDQwLTRlMzUtYjk2ZS04YzBiMTc0Y2RhODAiLCJpc3MiOiJodHRwczovL3JlZnJhY3QtbG9naW4uZm9zZm9yLmNvbS9hdXRoL3JlYWxtcy9tb3NhaWMiLCJhdWQiOlsibW9zYWljLWdhdGVrZWVwZXIiLCJhY2NvdW50Il0sInN1YiI6IjZjMjU4MWU3LWZmMTItNDljNy04MDJmLWI2ZjQzOWQxZDIwMSIsInR5cCI6IkJlYXJlciIsImF6cCI6Im1vc2FpYy1nYXRla2VlcGVyIiwic2Vzc2lvbl9zdGF0ZSI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJNTE9QUyIsImxvbmdfbGl2ZWRfdG9rZW4iLCJzcGVjdHJhLWRldmVsb3BlciIsImRlZmF1bHQtcm9sZXMtbW9zYWljIiwicmVmcmFjdC1kZXZlbG9wZXIiLCJvZmZsaW5lX2FjY2VzcyIsImFkbWluIiwidW1hX2F1dGhvcml6YXRpb24iLCJyZWZyYWN0LWFkbWluIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJvcGVuaWQgZW1haWwgcHJvZmlsZSIsInNpZCI6IjBhY2Y3YWZhLTVmMzMtNGRhZS05OGM3LTQyZDQwYTdlZTM2NiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJuYW1lIjoiUmVmcmFjdCBCRlNJIiwicHJlZmVycmVkX3VzZXJuYW1lIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20iLCJnaXZlbl9uYW1lIjoiUmVmcmFjdCIsImZhbWlseV9uYW1lIjoiQkZTSSIsImVtYWlsIjoicmVmcmFjdC5iZnNpQGZvc2Zvci5jb20ifQ.b6SYLgjo9Veo3GmJ8eZjCTNupQjpfMhzsoXdYjWwRtvRnNjBfx0gOqcugO9OcGn-mm8wwpSGI5uiL30-I6SdWBjsf1ur6GztoX7j-nP_3SrJJn3UhNNqIO8LbsPi5gGRTzWtnfjz92BF1YaCXxQwPY0P_aa8vJ6JxZz5Uctn9aIPIJZZnnjC_GPXtXurmshM_tEN2kwCjhEyr7wYzRqUoMtBGfpLjZREBzgZY-x6JyYiXNtycb1d6PFcCXf7nJVV8ienEC_x7OuciDzfeqd-SQnImvAHH7rqFdi9smBN08AbkDS2uAbMrokHrmbiBpaimrR013VwCWz2KL5QYlWleA"
request_payload_file = '/data/Magicoder_payloads.txt'
report_path = '/data/vLLM_5Concurrency_1gpu.csv'
df = pd.DataFrame(columns=['Response', 'Response_status', 'Time (seconds)'])

In [52]:
payload = {"prompt": "Problem: Write a function in python to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams.", 
           "max_tokens": 200,
           "temperature": 0,
           "model": "/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66"}

In [53]:
def read_payloads(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            yield line.strip()

In [54]:
def my_decorator(func):
    def calculate_execution_time(*args, **kwargs):
        start_time = time.time()
        result, df = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        data = {'Response':result.text, 'Response_status':result.status_code, 'Time (seconds)':f'{execution_time:.6f}'}
        print(data)
        df = df.append(data, ignore_index=True)
        print(f"Execution time: {execution_time:.6f} seconds \n\n")
        print(df)
        return result
    return calculate_execution_time

In [55]:
def save_report(file_path, df1):
    print(df1)
    file_exists = os.path.exists(file_path)
    df1.to_csv(file_path, index=False)

In [56]:
@my_decorator
def inference(payload_data=None, df = df):
    headers = {
        'Content-Type': 'application/json'
    }
    try:
        response = requests.request("POST", url, headers=headers, json=payload_data)
        print(f"Prompt==> {payload_data['prompt']} \n {response.text}")
        return response, df
    except Exception as e:
        print(e)
        

#Model Inference(Request)
if __name__ == "__main__":
    # Using the generator function to read lines  {**payload, 'prompt': {prompt}}
    prompts = read_payloads(request_payload_file)
    for i in range(1):
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(inference, {**payload, 'prompt': prompt}) for prompt in prompts]
            concurrent.futures.wait(futures)
    save_report(report_path, df)
#     a=inference(payload)

Prompt==> {"payload":"Problem: Write a function in python to reverse a given string.Input: String s.Output: String, the reversed version of s."} 
 {"id":"cmpl-38aab5f5a97340f28db3cf2ca1ce93a0","object":"text_completion","created":433859,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\n\ndef reverse_string(s):\n    return s[::-1]\n\ns = input(\"Enter a string: \")\nprint(\"Reversed string is: \", reverse_string(s))","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":36,"total_tokens":83,"completion_tokens":47}}
{'Response': '{"id":"cmpl-38aab5f5a97340f28db3cf2ca1ce93a0","object":"text_completion","created":433859,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\\n\\n\\ndef reverse_string(s):\\n    return s[::-1]\\n\\ns = input(\\"Enter a string: \\")\\nprint(\\"Reversed string is: \\", re

Prompt==> {"payload":"Problem: Write a function in java to perform binary search on a sorted list.Input: Sorted list arr and target value target.Output: Integer, the index of target in arr (or -1 if not found)."} 
 {"id":"cmpl-39a95aaa10284b44860ab4cd8a99e324","object":"text_completion","created":433863,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":52,"total_tokens":53,"completion_tokens":1}}
{'Response': '{"id":"cmpl-39a95aaa10284b44860ab4cd8a99e324","object":"text_completion","created":433863,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":52,"total_tokens":53,"completion_tokens":1}}', 'Response_status': 200, 'Time (seconds)': '0.118204'}
Execution time: 0.118204 seco

Prompt==> {"payload":"Problem: Write a function in c++ to count the number of words in a given string.Input: String s.Output: Integer, the number of words in s."} 
 {"id":"cmpl-b41f1d30c46341adb77b62c9d03aa18a","object":"text_completion","created":433865,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":42,"total_tokens":43,"completion_tokens":1}}
{'Response': '{"id":"cmpl-b41f1d30c46341adb77b62c9d03aa18a","object":"text_completion","created":433865,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":42,"total_tokens":43,"completion_tokens":1}}', 'Response_status': 200, 'Time (seconds)': '0.107282'}
Execution time: 0.107282 seconds 


                                           

Prompt==> {"payload":"Problem: Write a function in javascript to calculate the sum of the digits of a given positive integer.Input: Positive integer num.Output: Integer, the sum of the digits of num."} 
 {"id":"cmpl-0dba4df076844e9c954dc003abb479f3","object":"text_completion","created":433870,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":48,"total_tokens":49,"completion_tokens":1}}
{'Response': '{"id":"cmpl-0dba4df076844e9c954dc003abb479f3","object":"text_completion","created":433870,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":48,"total_tokens":49,"completion_tokens":1}}', 'Response_status': 200, 'Time (seconds)': '0.107953'}
Execution time: 0.107953 seconds 


    

Prompt==> {"payload":"Problem: Write a function in c++ to check if a given number is a prime number.Input: Positive integer num.Output: Boolean, indicating whether num is prime."} 
 {"id":"cmpl-a82df090e1f641569ad61e540611eca8","object":"text_completion","created":433863,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\nHere is a simple C++ function to check if a given number is prime:\n\n```cpp\n#include <iostream>\nusing namespace std;\n\nbool isPrime(int num) {\n    if (num <= 1) {\n        return false;\n    }\n    for (int i = 2; i * i <= num; i++) {\n        if (num % i == 0) {\n            return false;\n        }\n    }\n    return true;\n}\n\nint main() {\n    int num;\n    cout << \"Enter a number: \";\n    cin >> num;\n    if (isPrime(num)) {\n        cout << num << \" is a prime number.\" << endl;\n    } else {\n        cout << num << \" is not a prime number.\" << endl;\n    }\n  

...

Prompt==> {"payload":"Problem: Implement form validation in react to ensure that specific fields are filled out correctly.Input: Form input values.Output: Boolean, indicating whether the form is valid."} 
 {"id":"cmpl-bd51a3fc8dce439081e9281216cde213","object":"text_completion","created":433879,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":42,"total_tokens":43,"completion_tokens":1}}
{'Response': '{"id":"cmpl-bd51a3fc8dce439081e9281216cde213","object":"text_completion","created":433879,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":42,"total_tokens":43,"completion_tokens":1}}', 'Response_status': 200, 'Time (seconds)': '0.120260'}
Execution time: 0.120260 seconds 


  

Prompt==> {"payload":"Problem: Build a function in angular to toggle between light and dark modes for a website.Input: None.Output: None (update the website theme)."} 
 {"id":"cmpl-dc0e32a81d9346d6a1ed328808af27af","object":"text_completion","created":433879,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":38,"total_tokens":39,"completion_tokens":1}}
{'Response': '{"id":"cmpl-dc0e32a81d9346d6a1ed328808af27af","object":"text_completion","created":433879,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":38,"total_tokens":39,"completion_tokens":1}}', 'Response_status': 200, 'Time (seconds)': '0.118293'}
Execution time: 0.118293 seconds 


                                       

Prompt==> {"payload":"Problem: Implement an autocomplete feature in jQuery for a search input field.Input: String, user input in the search field.Output: None (update the autocomplete suggestions)."} 
 {"id":"cmpl-2989097b89a34e8e8ff86a5c4dcd629c","object":"text_completion","created":433880,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":48,"total_tokens":49,"completion_tokens":1}}
{'Response': '{"id":"cmpl-2989097b89a34e8e8ff86a5c4dcd629c","object":"text_completion","created":433880,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":48,"total_tokens":49,"completion_tokens":1}}', 'Response_status': 200, 'Time (seconds)': '0.122746'}
Execution time: 0.122746 seconds 


      

Prompt==> {"payload":"Problem: Implement a function in jQuery to validate an email address.Input: String, email address.Output: Boolean, indicating whether the email address is valid."} 
 {"id":"cmpl-58b7da326db74ac19f63d7fc7d0c9901","object":"text_completion","created":433880,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":40,"total_tokens":41,"completion_tokens":1}}
{'Response': '{"id":"cmpl-58b7da326db74ac19f63d7fc7d0c9901","object":"text_completion","created":433880,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":40,"total_tokens":41,"completion_tokens":1}}', 'Response_status': 200, 'Time (seconds)': '0.117152'}
Execution time: 0.117152 seconds 


                    

Prompt==> {"payload":"Problem: Write a function in Ruby to check if two strings are anagrams.Input: Strings s1 and s2.Output: Boolean, indicating whether s1 and s2 are anagrams."} 
 {"id":"cmpl-49e3a1893fbb4112b6d8f31998d4fa43","object":"text_completion","created":433876,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\n\ndef anagrams?(s1, s2)\n  s1.chars.sort == s2.chars.sort\nend\n\nputs anagrams?(\"listen\", \"silent\") # => true\nputs anagrams?(\"hello\", \"world\") # => false","logprobs":null,"finish_reason":"stop"}],"usage":{"prompt_tokens":51,"total_tokens":120,"completion_tokens":69}}
{'Response': '{"id":"cmpl-49e3a1893fbb4112b6d8f31998d4fa43","object":"text_completion","created":433876,"model":"/llmmodels/models--ise-uiuc--Magicoder-S-DS-6.7B/snapshots/cff055b1e110cbe75c0c3759bd436299c6d6bb66","choices":[{"index":0,"text":"\\n\\ndef anagrams?(s1, s2)\\n  s1.chars.sort == s2.chars.sort\\

In [ ]:
# import aiohttp
# import asyncio
# import time

# start_time = time.time()
# headers={"Content-type":"application/json"} 
# async def main():

#     async with aiohttp.ClientSession(trust_env = True, headers=headers) as session:

# #         for payload in payloads:
# #             url='https://refract.fosfor.com/openchatfinetunedimqlgenerator/d0721c53-c9f5-4d79-a988-2d21df46198f/score' 
#         async with session.post(url, json=payload) as resp:
#                 mql = await resp.json(content_type=None)
#                 print(mql)

# await main()
# print("--- %s seconds ---" % (time.time() - start_time))